In [1]:
from common import *

cursor = connect()

In [2]:
sql = """
SELECT 
  event_object_table AS table_name, 
  trigger_name 
FROM 
  information_schema.triggers 
GROUP BY 
  table_name, 
  trigger_name 
ORDER BY 
  table_name, 
  trigger_name;
"""

run_sql(cursor, sql)

       table_name           trigger_name
0           actor           last_updated
1         address           last_updated
2        category           last_updated
3            city           last_updated
4         country           last_updated
5        customer           last_updated
6            film  film_fulltext_trigger
7            film           last_updated
8      film_actor           last_updated
9   film_category           last_updated
10      inventory           last_updated
11       language           last_updated
12         rental           last_updated
13          staff           last_updated
14          store           last_updated


In [4]:
sql = """
SELECT 
  event_object_table AS table_name, 
  trigger_name 
FROM 
  information_schema.triggers 
WHERE 
  event_object_table = 'film' 
GROUP BY 
  table_name, 
  trigger_name 
ORDER BY 
  table_name, 
  trigger_name;
"""
run_sql(cursor, sql)

  table_name           trigger_name
0       film  film_fulltext_trigger
1       film           last_updated


## 创建一个用户自定义函数

In [5]:
sql = """
create or replace function get_triggers()
returns table (
	table_name text,
	trigger_name text
)
language plpgsql
as
$$
begin
	return query select 
	  trg.event_object_table::text AS table_name, 
	  trg.trigger_name::text
	from 
	  information_schema.triggers trg
	group by 
	  table_name, 
	  trg.trigger_name
	order by
	  table_name, 
	  trg.trigger_name;
end;
$$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1d00d5dd3d0>

In [6]:
sql = """
SELECT * FROM get_triggers();
"""
run_sql(cursor, sql)

       table_name           trigger_name
0           actor           last_updated
1         address           last_updated
2        category           last_updated
3            city           last_updated
4         country           last_updated
5        customer           last_updated
6            film  film_fulltext_trigger
7            film           last_updated
8      film_actor           last_updated
9   film_category           last_updated
10      inventory           last_updated
11       language           last_updated
12         rental           last_updated
13          staff           last_updated
14          store           last_updated


## 自定义函数增加参数

In [7]:
sql = """

create or replace function get_triggers(
	p_table_name text
)
returns table (
	table_name text,
	trigger_name text
)
language plpgsql
as
$$
begin
	return query select 
	  event_object_table::text AS table_name, 
	  trg.trigger_name::text
	from 
	  information_schema.triggers trg
	where 
	   event_object_table = p_table_name
	group by 
	  table_name, 
	  trg.trigger_name
	order by
	  table_name, 
	  trg.trigger_name;
end;
$$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1d00d5dd3d0>

In [9]:
sql = """
SELECT * FROM get_triggers('film');
"""
run_sql(cursor, sql)

  table_name           trigger_name
0       film  film_fulltext_trigger
1       film           last_updated


## 从 pg_trigger view 查询

|Column	        |Description|
|---------------|-----------|
|tgname	        |The name of the trigger.|
|tgrelid	    |The object ID of the table or view to which the trigger belongs.
|tgfoid	        |The object ID of the function is called when the trigger fires.
|tgtype	        |The type of the trigger, such as BEFORE, AFTER, or INSTEAD OF.
|tgenabled	    |The status of the trigger, either enabled or disabled.
|tgisinternal	|The boolean indicator indicates whether the trigger is a system-generated trigger or a user-defined trigger.


In [10]:
sql = """
SELECT 
  tgname AS trigger_name 
FROM 
  pg_trigger 
WHERE 
  tgrelid = 'film' :: regclass 
  AND tgisinternal = false 
ORDER BY 
  trigger_name;
"""
run_sql(cursor, sql)

            trigger_name
0  film_fulltext_trigger
1           last_updated
